# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../da…",0.983033,"""21209358239740…","""46181512000595…","""28477466700164…","""12724293801229…",false
"""../../../../da…",0.144221,"""93011921793412…","""14013650743584…","""28477466700164…","""14704814367988…",true
"""../../../../da…",0.679745,"""10987397888104…","""85532152597147…","""17834641723577…","""12724293801229…",false
"""../../../../da…",1.0,"""17740482247854…","""14013650743584…","""30755678553941…","""12724293801229…",false
"""../../../../da…",0.718982,"""49416424254604…","""46181512000595…","""30755678553941…","""12549311079532…",true
"""../../../../da…",0.0,"""11983042900609…","""85532152597147…","""10766864644593…",null,false
"""../../../../da…",0.02333,"""12501325239003…","""85532152597147…","""30755678553941…","""12549311079532…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""85763273867348…","""10083466760044…","""FirstMortgage3…","""17105498002303…","""Rejected"""
"""../../../../da…",0.144221,"""35505825081081…","""11668472835631…","""FirstMortgage3…","""64380982608184…","""Accepted"""
"""../../../../da…",0.679745,"""14584267168892…","""13980525220423…","""MoneyMarketSav…","""17105498002303…","""Rejected"""
"""../../../../da…",1.0,"""53167053567232…","""11668472835631…","""BasicChecking""","""17105498002303…","""Rejected"""
"""../../../../da…",0.718982,"""12375091005067…","""10083466760044…","""BasicChecking""","""16660927597664…","""Accepted"""
"""../../../../da…",0.0,"""12723453319523…","""13980525220423…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""18049397082125…","""13980525220423…","""BasicChecking""","""16660927597664…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""16936646699392…","""14630132741461…","""FirstMortgage3…","""57899048446866…","""Rejected"""
"""../../../../da…",0.144221,"""23234019987168…","""15549409961407…","""FirstMortgage3…","""11569888462450…","""Accepted"""
"""../../../../da…",0.679745,"""11723650837036…","""22364763783306…","""MoneyMarketSav…","""57899048446866…","""Rejected"""
"""../../../../da…",1.0,"""10056137171724…","""15549409961407…","""BasicChecking""","""57899048446866…","""Rejected"""
"""../../../../da…",0.718982,"""12886228061037…","""14630132741461…","""BasicChecking""","""17697903747526…","""Accepted"""
"""../../../../da…",0.0,"""13172422145582…","""22364763783306…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""96050003098023…","""22364763783306…","""BasicChecking""","""17697903747526…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""15825643450177…","""18084898034716…","""FirstMortgage3…","""86384298334016…","""Rejected"""
0.144221,"""55101795317837…","""21464700689903…","""FirstMortgage3…","""65467184100202…","""Accepted"""
0.679745,"""17867860379024…","""55191797887425…","""MoneyMarketSav…","""86384298334016…","""Rejected"""
1.0,"""42466269793261…","""21464700689903…","""BasicChecking""","""86384298334016…","""Rejected"""
0.718982,"""16437281485374…","""18084898034716…","""BasicChecking""","""18030301493602…","""Accepted"""
0.0,"""12099079633666…","""55191797887425…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""25585533347943…","""55191797887425…","""BasicChecking""","""18030301493602…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""c73023dda51de1…","""af486bc1815c72…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",0.144221,"""5414b197fb87b3…","""3cdf7b1ae1342e…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""../../../../da…",0.679745,"""892f5d61a9215e…","""806ef34eb8237f…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",1.0,"""c3aa807d9e927a…","""3cdf7b1ae1342e…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",0.718982,"""c2d492f9a05073…","""af486bc1815c72…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""../../../../da…",0.0,"""b21059221605c4…","""806ef34eb8237f…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""80cbe489dcecd4…","""806ef34eb8237f…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
